# Visualize Data 4

In this project I will try to predict protein binding residues using ProtT5 embeddings and distance maps computed from AF2 structures.
To better understand the data input and decide on an appropriate Machine Learning (ML) model, I am creating this notebook to visualize the data.
I am particularly interested in seeing if there are some obvious patterns first on the protein level and second on the residue level of the data.
Finally, I want to analyze some characteristics of binding sites such us binding site length and the 3D structure of binding sites.

In [1]:
# Import dependencies
%load_ext autoreload
%autoreload 2

import pandas as pd
from config import AppConfig
from dataset import Dataset
import plotly.express as px
from umap import UMAP
from visualize_data import umap_plots, umap_plots_2d
import numpy as np

## Read data

In [2]:
dataset = Dataset(config=AppConfig())

## Inspect residue level stats

I am extracting embedings and distograms for each residue. 
I am interested to find patterns in AA composition that could be used as predictors for binding residues and/or binding sites.

In [4]:
long_df, long_embeddings, long_labels, long_distograms = dataset.long_data()

Distogram length is different for id: Q00277. Seq length: 169, Distogram length: 168. Skipping...
Distogram length is different for id: P84801. Seq length: 121, Distogram length: 120. Skipping...
Distogram length is different for id: Q9NZV6. Seq length: 116, Distogram length: 115. Skipping...
Distogram length is different for id: C8BD48. Seq length: 99, Distogram length: 98. Skipping...


## Questions

### Would the filtering based on pLDDT affect the patterns?

### What is the pLDDT distribution of my data?

### What does the binding site structure look like in the data? How long are the binding sites? How far appart in sequence are their binding residues?

### This raises the question about the 3D structure of the binding sites. Do the binding residues reside close to each other in sequence? Or do the distograms acctually help bring distant binding residues (in sequence) together to form a single binding site?